# Goal
Build a simple VAE model based on the MNIST dataset.

1. Install dependencies
2. Select device

In [1]:
# Import libraries

import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.3.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/chelseaguo/Documents/MDST Text to Image Generator/F25-text-to-image-generator/venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/chelseaguo/Documents/MDST Text to Image Generator/F25-text-to-image-generator/venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users

In [3]:
# Device selection

if torch.cuda.is_available():
    print("Using GPU")
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    print("Using MPS")
    device = torch.device("mps")
else:
    print("Using CPU")
    device = torch.device("cpu")

kwargs = {'num_workers': 1}
if torch.cuda.is_available():
    kwargs['pin_memory'] = True

Using CPU


In [4]:
# Hyperparameters

batch_size = 64
latent_size = 32
init_channels = 8
epochs = 10

In [5]:
# Download the MNIST dataset

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        'data', train=True, download=True,
        transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(
        'data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=False, **kwargs)

Failed to download (trying next):
HTTP Error 404: Not Found

Failed to download (trying next):
<urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)>



RuntimeError: Error downloading train-images-idx3-ubyte.gz

In [ ]:
class VAE(nn.Module):
    def __init__(self, image_channels, init_channels, latent_size):
        super(VAE, self).__init__()
        
        # encoder
        self.encoder = nn.Sequential(
            nn.Conv2d(image_channels, init_channels, kernel_size = 4, stride = 2, padding = 1),
            nn.ReLU(),
            nn.Conv2d(init_channels, init_channels * 2, kernel_size = 4, stride = 2, padding = 1),
            nn.ReLU(),
            nn.Conv2d(init_channels * 2, init_channels * 4, kernel_size = 4, stride = 2, padding = 1),
            nn.ReLU(),
            nn.Conv2d(init_channels * 4, init_channels * 8, kernel_size = 4),
            nn.ReLU()
        )
        
        # FC layers to get mu and logvar
        self.fc1 = nn.Linear(64, 128)
        self.fc_mu = nn.Linear(128, latent_size)
        self.fc_logvar = nn.Linear(128, latent_size)
        self.fc2 = nn.Linear(latent_size, 64)
        
        # decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(init_channels * 8, init_channels * 4, kernel_size = 4),
            nn.ReLU(),
            nn.ConvTranspose2d(init_channels * 4, init_channels * 2, kernel_size = 4, stride = 2, padding = 1),
            nn.ReLU(),
            nn.ConvTranspose2d(init_channels * 2, init_channels, kernel_size = 4, stride = 2, padding = 1),
            nn.ReLU(),
            nn.ConvTranspose2d(init_channels, image_channels, kernel_size = 4, stride = 2, padding = 1),
            nn.ReLU(),
            nn.Conv2d(1, 1, kernel_size = 5, padding = 2),
            nn.Sigmoid()
        )
    
    def encode(self, x):
        h = self.encoder(x)
        h = h.view(h.size(0), -1)        
        h_fc = F.relu(self.fc1(h))
        mu = self.fc_mu(h_fc)
        logvar = self.fc_logvar(h_fc)
        return mu, logvar
    
    # mu: mean vector
    # logvar: log variance vector
    def reparameterize(self, mu, logvar):
        # TODO: Define the reparameterization function
        
        return None  # Delete this line before you begin
    
    def decode(self, z):
        h = F.relu(self.fc2(z))
        h = h.view(-1, 64, 1, 1)
        
        return torch.sigmoid(self.decoder(h))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        recon_x = self.decode(z)
        return recon_x, mu, logvar

In [ ]:
# recon_x: reconstructed images
# x: original input images
# mu: mean vector
# logvar: log variance vector
def loss_function(recon_x, x, mu, logvar):
    # TODO: Define the reconstruction loss(binary cross entropy) term and KL divergence term
    
    return None     # Delete this line before you begin

In [ ]:
# Train and Test functions
def train(model, optimizer, epoch, losses):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        recon_batch, mu, logvar = model(data)
        optimizer.zero_grad()
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.detach().cpu().numpy()
        optimizer.step()
        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))
    
    losses.append(train_loss / len(train_loader.dataset))
    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

def test(model, epoch, losses):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).detach().cpu().numpy()
            if i == 0:
                n = min(data.size(0), 5)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(-1, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'reconstruction_' + str(f"{epoch:02}") + '.png', nrow=n)

    losses.append(test_loss / len(test_loader.dataset))
    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [ ]:
model = VAE(image_channels=1, init_channels=init_channels, latent_size=latent_size).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

train_losses = []
test_losses = []

for epoch in range(1, epochs + 1):
    train(model, optimizer, epoch, train_losses)
    test(model, epoch, test_losses)
    with torch.no_grad():
        sample = torch.randn(10, latent_size).to(device)
        sample = model.decode(sample).to(device)
        save_image(sample.view(10, 1, 28, 28), str(f"sample_{epoch:02}.png"))

In [ ]:
# Visualize the training and test losses
import matplotlib.pyplot as plt
print(train_losses)
print(test_losses)
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Test Losses')
plt.grid()
plt.legend()
plt.show()

In [ ]:
# Save the model
import torchvision.models as models

torch.save(model.state_dict(), 'vae_model_mnist.pth' )

In [ ]:
# Load the model

model = VAE(image_channels=1, init_channels=init_channels, latent_size=latent_size).to(device)
model.load_state_dict(torch.load('vae_model_mnist.pth'))

In [ ]:
# Test the model

def generate_digit(model, num_samples=1):
    model.eval()
    with torch.no_grad():
        sample = torch.randn(num_samples, latent_size).to(device)
        sample = model.decode(sample).to(device)
        
        save_image(sample.view(num_samples, 1, 28, 28), f"generated_sample.png")

generate_digit(model, num_samples=10)